# Segmenting and Clustering Neighborhoods in Toronto

### Part 1 - Retrieving The Toronto Neightborhoods Data

We start by getting the necessary data from Wikipedia.  
[List of Postal Codes of Canada - M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [1]:
#!conda install -c anaconda beautifulsoup4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium

In [2]:
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)

soup = BeautifulSoup(req.content, 'html.parser')

table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Now, we have the data all ready for some pre-processing.  
Let's drop unassigned boroughs.

In [3]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
print(df.shape)

(103, 3)


### Part 2 - Retrieve GPS Coordinates of the Boroughs

Getting the coordinates of Toronto first!

In [5]:
address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.8184944, -79.3309468.


I could not get geolocator to work at all. However, the Tomtom API offers the possibility to search a postcode with optional arguments to make sure we end up in Toronto! 

Note the CA country code to narrow down the search area and not return a location with the same post code in a different country!  

I removed my key but it is free and easy to get one on TomTom API website.

In [6]:
def findCoordinates(target):
    try:
        url = "https://api.tomtom.com/search/2/geocode/"
        url2 = ".json?limit=1&lat=43.7&lon=-79.3&radius=50000&key=XXXX"
        results = requests.get(url + target + url2).json()
        latitude = results['results'][0]['position']['lat']
        longitude = results['results'][0]['position']['lon']
        return [latitude, longitude]
    except:
        return [0, 0]

I was initially missing lots of data in a periodic pattern (every 10 rows or so), so I added a short pause between each request to the API. This worked quite well.

In [7]:
import time
data = {'latitude': [], 'longitude': []}

for target in df['Postal Code']:
    
    [long, lat] = findCoordinates(target)
    time.sleep(0.05)
    
    data['latitude'].append(long)
    data['longitude'].append(lat)

GPS = pd.DataFrame.from_dict(data)

In [8]:
df = pd.concat([df, GPS], axis=1)
pd.set_option('display.max_rows', 101)
df.head(101)

,Postal Code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.75612,-79.32964
1,M4A,North York,Victoria Village,43.72678,-79.31074
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65535,-79.36505
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72200,-79.44591
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66391,-79.38873
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66452,-79.52908
6,M1B,Scarborough,"Malvern, Rouge",43.80708,-79.20155
7,M3B,North York,Don Mills,43.74631,-79.35760
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70668,-79.30658
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65748,-79.37863


We are missing just a data point, which is a processing centre which we are not really interested in for our future home neighborhood! We can drop this one.  
After this, we save the dataframe as a CSV so that we don't need to send 100 requests to the API everytime we want to reload the data.

In [9]:
df.drop(76, inplace = True)
#df.to_csv(r'TorontoNeighborhoods.csv')
#df = pd.read_csv("TorontoNeighborhoods.csv", index_col=0)

Let's have a first look at our neighborhoods!  
This function plots the GPS coordinates of each neighborhoods, and allocates a specific color based on the digit in the postal code.

In [10]:
def plot_Toronto(df, group):
    
    [latitude, longitude] = [43.7, -79.3]
    colors = ['red','blue','gray','darkred','orange','black','green','purple', 'yellow']

    venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the Conrad Hotel

    for lat, lng, label in zip(df.latitude, df.longitude, group):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color=colors[int(label)-1],
            popup=label,
            fill = True,
            fill_color=colors[int(label)-1],
            fill_opacity=0.6
        ).add_to(venues_map)

    return venues_map

plot_Toronto(df, df['Postal Code'].str[1])

### Part 3 - Clustering

We will only pick the Boroughs that contain "Toronto" in their names.

In [11]:
df_Downtown = df[df['Borough'].str.contains("Toronto")==1]
pd.set_option('display.max_rows', 15)
df_Downtown.reset_index(drop=True, inplace=True)
df_Downtown

,Postal Code,Borough,Neighborhood,latitude,longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65535,-79.36505
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66391,-79.38873
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65748,-79.37863
3,M5C,Downtown Toronto,St. James Town,43.65111,-79.37574
4,M4E,East Toronto,The Beaches,43.67656,-79.29416
...,...,...,...,...,...
34,M5W,Downtown Toronto,Stn A PO Boxes,43.64575,-79.37778
35,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.66793,-79.37020
36,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.64858,-79.38172
37,M4Y,Downtown Toronto,Church and Wellesley,43.66629,-79.38245


Now we define the Foursquare credentials.

In [12]:
CLIENT_ID = 'XXX' # your Foursquare ID
CLIENT_SECRET = 'XXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TYN2NSLD2H3O33G1GKM3KSWIXTN40P0NNWHSBKCTKCPTFNBV
CLIENT_SECRET:3D0B3UI55ZIROTX3LC5JFI3XCE5ZHJXJ0GSQ05V3YINQQUCL


We create a function to fetch the venues.

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We now run the function for each of our neighborhoods.

In [14]:
LIMIT = 100
radius = 500

toronto_venues = getNearbyVenues(names=df_Downtown['Neighborhood'],
                                   latitudes=df_Downtown['latitude'],
                                   longitudes=df_Downtown['longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(1871, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65535,-79.36505,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
1,"Regent Park, Harbourfront",43.65535,-79.36505,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65535,-79.36505,Tandem Coffee,43.653559,-79.361809,Coffee Shop
3,"Regent Park, Harbourfront",43.65535,-79.36505,Berkeley Church,43.655123,-79.365873,Event Space
4,"Regent Park, Harbourfront",43.65535,-79.36505,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


Now is time to start preparing the clustering by one hot encoding the data.

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_onehot.shape

(1871, 233)

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.010526,0.000000,0.0,0.010526,0.010526,0.021053,0.000000,0.000000,0.00,...,0.000000,0.00,0.0,0.010526,0.000000,0.0,0.000000,0.0,0.0,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.017857,0.0,0.000000,0.000000,0.017857,0.000000,0.035714,0.00,...,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.018868,0.000000,0.0,0.000000,0.000000,0.018868,0.000000,0.000000,0.00,...,0.000000,0.00,0.0,0.018868,0.000000,0.0,0.000000,0.0,0.0,0.00
4,Central Bay Street,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.017241,0.000000,0.00,...,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.017241,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,"The Annex, North Midtown, Yorkville",0.000000,0.000000,0.0,0.040000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.0,0.040000,0.000000,0.0,0.000000,0.0,0.0,0.00
35,The Beaches,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.250000,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00
36,"The Danforth West, Riverdale",0.020833,0.000000,0.0,0.020833,0.000000,0.000000,0.000000,0.000000,0.00,...,0.020833,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00
37,"Toronto Dominion Centre, Design Exchange",0.000000,0.000000,0.0,0.040000,0.000000,0.010000,0.000000,0.000000,0.03,...,0.000000,0.01,0.0,0.010000,0.000000,0.0,0.010000,0.0,0.0,0.01


In [21]:
toronto_grouped.shape

(39, 233)

Let's get an overview of the top 5 venues for each neighborhood.

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2  Japanese Restaurant  0.04
3   Italian Restaurant  0.04
4         Cocktail Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.11
1          Bar  0.07
2  Coffee Shop  0.05
3       Lounge  0.05
4   Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Fast Food Restaurant  0.16
1                  Park  0.11
2       Harbor / Marina  0.11
3             Pet Store  0.05
4            Restaurant  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.06
2          Restaurant  0.06
3               Hotel  0.06
4                 Gym  0.04


----Central Bay Street----
                 venue  freq
0      

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Seafood Restaurant,Breakfast Spot,Cocktail Bar,Beer Bar,Restaurant,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Gift Shop,Lounge,Coffee Shop,Restaurant,Bakery,Arts & Crafts Store,Thrift / Vintage Store,Cocktail Bar
2,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Park,Harbor / Marina,Brewery,Movie Theater,Farmers Market,Liquor Store,Skate Park,Steakhouse,Sandwich Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Hotel,Restaurant,Italian Restaurant,Grocery Store,Gym,Park,Beer Bar,Pub,Peruvian Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Bar,Bubble Tea Shop,Park,Dessert Shop,Indian Restaurant


#### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 10 clusters.

In [35]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 9, 8, 5, 5, 9, 5, 5, 1, 1])

In [36]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = df_Downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [40]:
toronto_merged.head(20) # check the last columns!

,Postal Code,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65535,-79.36505,9,Coffee Shop,Breakfast Spot,Italian Restaurant,Yoga Studio,Bakery,Gym / Fitness Center,Diner,Discount Store,Electronics Store,Sandwich Place
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66391,-79.38873,9,Coffee Shop,College Cafeteria,Gym,Yoga Studio,Dance Studio,Burrito Place,Café,Ramen Restaurant,Park,Creperie
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65748,-79.37863,1,Coffee Shop,Clothing Store,Italian Restaurant,Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Bubble Tea Shop,Lingerie Store
3,M5C,Downtown Toronto,St. James Town,43.65111,-79.37574,5,Coffee Shop,Café,Cocktail Bar,Seafood Restaurant,Gym,American Restaurant,Restaurant,Italian Restaurant,Gastropub,Creperie
4,M4E,East Toronto,The Beaches,43.67656,-79.29416,0,Health Food Store,Pub,Trail,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,M4L,East Toronto,"India Bazaar, The Beaches West",43.67120,-79.31649,9,Indian Restaurant,Grocery Store,Sandwich Place,Café,Gym,Pet Store,Diner,Discount Store,Fish & Chips Shop,Snack Place
16,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.64878,-79.37896,5,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Japanese Restaurant,Gym,American Restaurant,Gastropub,Seafood Restaurant
17,M4M,East Toronto,Studio District,43.66182,-79.34218,9,Café,Coffee Shop,American Restaurant,Gym / Fitness Center,Gastropub,Bar,Sandwich Place,Bakery,Yoga Studio,Lounge
18,M4N,Central Toronto,Lawrence Park,43.72716,-79.39704,7,Park,Pool,Bus Line,Metro Station,Supermarket,Bank,Gastropub,Clothing Store,Spa,Cupcake Shop


In [38]:
import matplotlib.cm as cm
import matplotlib.colors as colors

[latitude, longitude] = [43.7, -79.35]
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The clusters tend to be correlated with their position. This can be expected as similar venues tend be located in the same area (example: social venues downtown or grocery stores in periphery). 